In [3]:
import tensorflow as tf
import numpy as np

In [4]:
x = tf.Variable(3.0)
with tf.GradientTape(persistent=True) as g:    
    # Grabar las operacioes en el contexto de GradientTape
    y = x**2

En este caso Y es la variable objetivo

In [5]:
dy_dx = g.gradient(y,x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


Un módulo es un contenedor con nombre para variables, otros módulos y funciones que se aplican a los datos del usuario.
Se ejecuta init al inicar una instancia, y call cuando esta es ejectudada.

In [6]:
# Definición del un modelo de regresión linear simple
class SimpleLinearRegression(tf.Module):
    def __init__(self, input_dim = 1, name=None):
        super().__init__(name=name)
        #Definir e inicializar parámetros
        rand_w = tf.random.normal([input_dim])
        rand_b = tf.random.normal([1])
        self.w = tf.Variable(rand_w, name='w', dtype=tf.float32, trainable=True)
        self.b = tf.Variable(rand_b, name='b', dtype=tf.float32, trainable=True)

    def __call__(self, x):
        y = tf.multiply(x, self.w) + self.b
        return y


In [7]:
#Creaion de un modelo de regresión lineal
lin_reg = SimpleLinearRegression(input_dim=1, name="Mi_Modelo")
# En este momento no se ha entrenado

#Utilcemos los datos del ejercicio anterior de predicción de masas
tiempo = tf.constant([5,7,12,16,20], dtype=tf.float32)
masa = tf.constant([40,120,180,210,240], dtype= tf.float32)


Yp = lin_reg(tiempo.numpy()) # Yp = Y predicha
print(Yp)

tf.Tensor([ -8.301536 -11.801718 -20.552172 -27.552536 -34.552902], shape=(5,), dtype=float32)


Retomamos la función de costo y calcular los gradientes respecto a un modelo

In [8]:
# print(lin_reg.trainable_variables)

# Función de costo MSE
def mse_loss(Y, Yp):
    e = Y - Yp
    se = tf.multiply(e,e)
    mse = tf.reduce_mean(se, axis=0)
    return mse

#loss = mse_loss(masa, r)
#print(loss)

In [9]:
# Cálculo del gradiente con respecto a un 
learining_rate = 0.0001

with tf.GradientTape() as tape:
    # Paso 1: Predicción
    Yp = lin_reg(tiempo)
    # Paso 2: Calcular Costo
    loss = mse_loss(masa, Yp)
    print(f"w0:{lin_reg.w.numpy()}, b: {lin_reg.b.numpy()}, costo: {loss}\n")
    # Paso 3: Calcular el gradiente
    gradients = tape.gradient(loss, lin_reg.trainable_variables)
    # Aplicar (regla de actualización de) los gradientes en la actualización de parámetros
    for g, v in zip(gradients, lin_reg.trainable_variables):
        v.assign_sub(learining_rate*g) # One Step

    print(f"w0:{lin_reg.w.numpy()}, b: {lin_reg.b.numpy()}\n")


    Yp = lin_reg(tiempo)
    loss = mse_loss(masa, Yp)
    print(f"w0:{lin_reg.w.numpy()}, b: {lin_reg.b.numpy()}, costo: {loss}\n")

w0:[-1.750091], b: [0.44891912], costo: 38347.28125

w0:[-1.2355852], b: [0.48462954]

w0:[-1.2355852], b: [0.48462954], costo: 35734.08203125



Min Cost =~ 485

w0 & b = 12.7428 & 1.834

N =~ 50?

Posibles Problemas:

- Mínimos Locales
- Saddle Points (Derivada ~= 0)
- Mismos parámetros nos llevarán por la misma dirección
- Los gradientes que calculamos pueden tener ruido.


Resolver Sobreentrnamiento:
- Batch
- Mini batch
- Forma Estocástica